# Logistic Regression

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
!ls

cifar10.ipynb cifar_net.pth data          fmnist.ipynb  mnist.ipynb


In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [8]:
use_mps = torch.backends.mps.is_available()
device = torch.device('mps' if use_mps else 'cpu')
model = nn.Linear(input_size, num_classes).to(device)

In [9]:
criterion = nn.CrossEntropyLoss()  #used with linear output. nll is paired with softmax act at out.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [10]:
#train
model.train()
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2347
Epoch [1/5], Step [200/600], Loss: 2.1419
Epoch [1/5], Step [300/600], Loss: 2.0190
Epoch [1/5], Step [400/600], Loss: 1.9189
Epoch [1/5], Step [500/600], Loss: 1.8591
Epoch [1/5], Step [600/600], Loss: 1.7623
Epoch [2/5], Step [100/600], Loss: 1.6987
Epoch [2/5], Step [200/600], Loss: 1.6702
Epoch [2/5], Step [300/600], Loss: 1.6237
Epoch [2/5], Step [400/600], Loss: 1.5532
Epoch [2/5], Step [500/600], Loss: 1.5135
Epoch [2/5], Step [600/600], Loss: 1.5104
Epoch [3/5], Step [100/600], Loss: 1.3992
Epoch [3/5], Step [200/600], Loss: 1.3885
Epoch [3/5], Step [300/600], Loss: 1.3234
Epoch [3/5], Step [400/600], Loss: 1.2412
Epoch [3/5], Step [500/600], Loss: 1.3231
Epoch [3/5], Step [600/600], Loss: 1.3018
Epoch [4/5], Step [100/600], Loss: 1.2082
Epoch [4/5], Step [200/600], Loss: 1.2837
Epoch [4/5], Step [300/600], Loss: 1.2118
Epoch [4/5], Step [400/600], Loss: 1.2367
Epoch [4/5], Step [500/600], Loss: 1.0584
Epoch [4/5], Step [600/600], Loss:

In [11]:
#test
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 82.73999786376953 %
